In [1]:
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
from tqdm import tqdm

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_score, recall_score, f1_score
from datetime import datetime

2024-01-08 23:44:02.288773: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-08 23:44:02.312659: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 23:44:02.312725: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 23:44:02.312757: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 23:44:02.318636: I tensorflow/core/platform/cpu_feature_g

In [2]:
SIZE=48
BATCH_SIZE=512

In [3]:
class UTKFaceDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.files = glob.glob(os.path.join(directory, '*.jpg'))
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_name = self.files[idx]
        image = Image.open(img_name)
        filename = img_name.split('/')[-1]
        sex = int(filename.split('_')[1])  # Предполагается, что имя файла начинается с возраста

        if self.transform:
            image = self.transform(image)

        return image, sex

In [4]:
transform = transforms.Compose([
    transforms.Resize((SIZE, SIZE)),
    transforms.ToTensor(),
])

dataset = UTKFaceDataset(directory='../data/UTKFace_48', transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
class SexEstimatorModel(nn.Module):
    def __init__(self):
        super(SexEstimatorModel, self).__init__()
        self.base_model = timm.create_model('efficientnetv2_rw_s', pretrained=True)
        self.base_model.classifier = nn.Linear(self.base_model.classifier.in_features, 1)
        
    def forward(self, x):
        x = self.base_model(x)
        return torch.sigmoid(x).squeeze()  # Используйте сигмоиду для бинарной классификации


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SexEstimatorModel().to(device)
loss_fun = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [7]:
writer = SummaryWriter(log_dir="log/sex", filename_suffix=datetime.now().strftime("%Y%m%d-%H%M%S"))

for epoch in tqdm(range(100)):  # проход по датасету несколько раз
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fun(outputs.view(-1), labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        writer.add_scalar('Metrics/epoch_loss', running_loss  / len(train_loader), epoch)
    
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for i, (images, labels) in enumerate(test_loader):
            images, labels = images.to(device), labels
            outputs = model(images)
            preds = outputs.round()
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.tolist())
    precision = precision_score(all_labels, all_preds, average="weighted")
    recall = recall_score(all_labels, all_preds, average="weighted")
    f1 = f1_score(all_labels, all_preds, average="weighted")
    writer.add_scalar('Metrics/precision', precision, epoch)
    writer.add_scalar('Metrics/recall', recall, epoch)
    writer.add_scalar('Metrics/f1', f1, epoch)
print('Finished Training')

100%|██████████| 100/100 [23:39<00:00, 14.19s/it]

Finished Training


In [8]:
torch.save(model, '../../models/sex_model_torch.pth')
torch.save(model.state_dict(), '../../models/sex_model_weights.pth')

In [11]:
# model = SexEstimatorModel()  # Создайте экземпляр вашей модели
# model.load_state_dict(torch.load('../../sex_model_torch.pth'))

model = torch.load('../../models/sex_model_torch.pth')

model.eval()  # Переведите модель в режим оценки

# Загрузите изображение
image_path = '../data/face_recognition_images/person1.1.jpg'
image = Image.open(image_path)

# Примените преобразования к изображению
image = transform(image)
image = image.to(device)
image = image.unsqueeze(0)  # Добавьте дополнительное измерение, так как модель ожидает пакет изображений

# Сделайте предсказание
with torch.no_grad():
    output = model(image)
    predicted_sex = output.round().item()  # Получите предсказанный возраст как число

print(f'Predicted Sex: {predicted_sex}')

Predicted Sex: 0.0
